In [1]:
# Cell 0: 모듈 import를 위한 경로 설정
import os, sys
sys.path.append(os.path.abspath(".."))  # shared, models 디렉토리 접근 가능하도록 경로 추가


In [2]:
# Cell 1: 환경 확인
import torch

print(f"✅ PyTorch version: {torch.__version__}")
print(f"🚀 GPU available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print(f"🧠 GPU name: {torch.cuda.get_device_name(0)}")
    
    total_memory = torch.cuda.get_device_properties(device).total_memory / 1024**3  # GiB
    reserved_memory = torch.cuda.memory_reserved(device) / 1024**3  # GiB
    allocated_memory = torch.cuda.memory_allocated(device) / 1024**3  # GiB
    free_memory = reserved_memory - allocated_memory  # GiB

    print(f"💾 Total memory: {total_memory:.2f} GiB")
    print(f"📦 Reserved memory: {reserved_memory:.2f} GiB")
    print(f"📈 Allocated memory: {allocated_memory:.2f} GiB")
    print(f"🟢 Free memory in reserved: {free_memory:.2f} GiB")


✅ PyTorch version: 2.6.0+cu124
🚀 GPU available: True
🧠 GPU name: Quadro RTX 5000
💾 Total memory: 15.73 GiB
📦 Reserved memory: 0.00 GiB
📈 Allocated memory: 0.00 GiB
🟢 Free memory in reserved: 0.00 GiB


In [3]:
# Cell 2: 데이터셋 로딩
from torch.utils.data import DataLoader
from shared.data_loader import HDF5Dataset
import os

input_dir = "/caefs/data/IllustrisTNG/subcube/input"
output_dir = "/caefs/data/IllustrisTNG/subcube/output"

input_files = sorted([os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith(".h5")])
output_files = sorted([os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.endswith(".h5")])

dataset = HDF5Dataset(input_files, output_files)
loader = DataLoader(dataset, batch_size=2, shuffle=True)

x, y = next(iter(loader))
print(f"✅ Sample loaded: input shape = {x.shape}, output shape = {y.shape}")


2025-07-30 20:46:57,291 | INFO | data_loader | 🔍 Initializing dataset with 12 file pairs.
2025-07-30 20:46:57,315 | INFO | data_loader | 📦 Total samples across all files: 110592


✅ Sample loaded: input shape = torch.Size([2, 1, 60, 60, 60]), output shape = torch.Size([2, 1, 60, 60, 60])


In [ ]:
from models.fno.model import FNO
import torch
from torchinfo import summary
import torch.nn as nn

# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# FNO 모델 초기화
model = FNO(
    in_channels=1,
    out_channels=1,
    modes1=32,
    modes2=32,
    modes3=32,
    width=128,
    lifting_channels=128,
    add_grid=True,
    activation=nn.ReLU
).to(device)

model.train()
print("✅ FNO model loaded and set to training mode.")

In [ ]:
summary(model, input_size=(2, 1, 60, 60, 60),
        col_names=["input_size", "output_size", "num_params", "kernel_size"])


In [ ]:
def test_batch_size(batch_size):
    try:
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
        model.eval()
        with torch.no_grad():
            for x, y in loader:
                x, y = x.to(device), y.to(device)
                _ = model(x)
                print(f"✅ Success with batch_size={batch_size}")
                break
    except RuntimeError as e:
        print(f"❌ Failed with batch_size={batch_size}: {str(e).splitlines()[0]}")

for bs in [32, 16, 8, 4, 2, 1]:
    test_batch_size(bs)


In [ ]:
from shared.losses import mse_loss, spectral_loss

loss_val = mse_loss(x.to(device), y.to(device))
print(f"✅ MSE Loss on sample batch: {loss_val.item():.4f}")


In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
print("✅ Optimizer and LR scheduler initialized.")


In [ ]:
from tqdm import tqdm

model.train()
loader = DataLoader(dataset, batch_size=4, shuffle=True)
n_batch = 10

for epoch in range(3):
    total_loss = 0.0
    for i, (inputs, targets) in enumerate(tqdm(loader, desc=f"Epoch {epoch+1}")):
        if i >= n_batch:
            break
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = mse_loss(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    scheduler.step()
    print(f"📉 Epoch {epoch+1} Loss: {total_loss / n_batch:.4f} | LR: {scheduler.get_last_lr()[0]:.2e}")


In [ ]:
# Cell 4: Trainer 설정 (디버깅용)
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.loggers import CSVLogger

trainer = Trainer(
    max_epochs=2,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    log_every_n_steps=1,
    enable_progress_bar=True,
    detect_anomaly=True,
    logger=CSVLogger("logs/debug_fno", name="fno_test"),
    callbacks=[
        EarlyStopping(monitor="val_loss", patience=2, mode="min", verbose=True)
    ]
)

In [ ]:
save_path = "fno_test_model.pt"
torch.save(model.state_dict(), save_path)
print(f"✅ FNO model saved to {save_path}")

state_dict = torch.load(save_path, map_location='cpu')
print(f"🔍 저장된 키 개수: {len(state_dict)}")
print("예시 키:", list(state_dict.keys())[:5])


In [ ]:
import matplotlib.pyplot as plt

model.eval()
with torch.no_grad():
    input_sample, target_sample = next(iter(loader))
    input_sample, target_sample = input_sample.to(device), target_sample.to(device)
    pred_sample = model(input_sample)




In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

# 예시: input = [B, 1, D, H, W] 중에서 B=0, D=30인 2D 슬라이스
input_slice = inputs[0, 0, 30].detach().cpu().numpy()
target_slice = targets[0, 0, 30].detach().cpu().numpy()
pred_slice = outputs[0, 0, 30].detach().cpu().numpy()

# ✅ log-transform (0을 피하기 위해 log(1 + x) 사용)
input_log = np.log10(1 + input_slice)

# 📊 시각화
fig, axs = plt.subplots(1, 3, figsize=(15, 4))

axs[0].imshow(input_log, cmap='viridis')
axs[0].set_title('Input (log10)')

axs[1].imshow(target_slice, cmap='viridis')
axs[1].set_title('Target')

axs[2].imshow(pred_slice, cmap='viridis')
axs[2].set_title('Prediction')

for ax in axs:
    ax.axis('off')

plt.tight_layout()
plt.show()
